#### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("GROQ_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true" ## Tracing of Langchain
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader=WebBaseLoader("https://docs.smith.langchain.com/evaluation")
loader

In [4]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/evaluation', 'title': 'Evaluation Quick Start | 🦜️🛠️ LangSmith', 'description': "Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.", 'language': 'en'}, page_content='\n\n\n\n\nEvaluation Quick Start | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationQuick StartTutorialsEvaluate a chatbotEvaluate a RAG applicationRun backtests on a new version of an agentRunning SWE-bench with LangSmithEvaluate a complex agentTest a ReA

In [5]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
documents=text_splitter.split_documents(docs)

In [6]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/evaluation', 'title': 'Evaluation Quick Start | 🦜️🛠️ LangSmith', 'description': "Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.", 'language': 'en'}, page_content='Evaluation Quick Start | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/evaluation', 'title': 'Evaluation Quick Start | 🦜️🛠️ LangSmith', 'description': "Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a s

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\aswan\AGen ai\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [9]:
vectorstoredb


In [10]:
## Query From a vector db
query="Evaluations are a quantitative way to measure"
result=vectorstoredb.similarity_search(query)
result[0].page_content

"Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.\nEvaluations are made up of three components:"

In [11]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="llama3-8b-8192",
               openai_api_base="https://api.groq.com/openai/v1",
               openai_api_key=os.environ["OPENAI_API_KEY"]
               )
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001D22C262120> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D22C262BA0> root_client=<openai.OpenAI object at 0x000001D22B077230> root_async_client=<openai.AsyncOpenAI object at 0x000001D22C262900> model_name='llama3-8b-8192' model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [12]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D22C262120>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D22C262BA0>, root_client=<openai.OpenAI object at 0x000001D22B077230>, root_async_client=<openai.AsyncOpenAI object at 0x000001D22C262900>, model_name='llama3-8b-8192', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":" ",
    "context":[Document(page_content="Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.")]
})

'What is the primary purpose of evaluations in Large Language Model (LLM) applications?\n\nAnswer: Evaluations are a quantitative way to measure the performance of LLM applications, providing a structured way to identify failures, compare changes across different versions, and build more reliable AI applications.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [14]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [15]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [16]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D207EADA90>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
         

In [21]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"Evaluations are a quantitative way to measure performance of LLM applications"})
response['answer']

'What is the key to defining custom evaluators for LLM applications?\n\nAccording to the context, the answer is: "you can define completely custom evaluators using your own code."'

In [22]:

response

{'input': 'Evaluations are a quantitative way to measure performance of LLM applications',
 'context': [Document(id='dcf53db1-dc48-452e-9fa5-4f0256b8b095', metadata={'source': 'https://docs.smith.langchain.com/evaluation', 'title': 'Evaluation Quick Start | 🦜️🛠️ LangSmith', 'description': "Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.", 'language': 'en'}, page_content="Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compar

In [23]:
response['context']

[Document(id='dcf53db1-dc48-452e-9fa5-4f0256b8b095', metadata={'source': 'https://docs.smith.langchain.com/evaluation', 'title': 'Evaluation Quick Start | 🦜️🛠️ LangSmith', 'description': "Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.", 'language': 'en'}, page_content="Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.\nEval